In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [30]:
bnf_pandas = pd.read_csv("../data/bnf.csv")

# INDEX = 'MIDCP'
# INDEX = 'BNF'
# INDEX = 'NIFTY'
# INDEX = 'FNF'
# INDEX = 'STOCK'
INDEX = 'SENSEX'

if INDEX == 'BNF':
    lev_ = 5.066
    slippage_ = 0.0001
    lot_size_ = 15
elif INDEX == 'NIFTY' or INDEX == 'FNF':
    lev_ = 6.333
    slippage_ = 0.0002
    lot_size_ = 25
elif INDEX == 'MIDCP':
    lev_ = 6.333
    slippage_ = 0.0005
    lot_size_ = 50
elif INDEX == 'SENSEX':
    lev_ = 7
    slippage_ = 0.0001
    lot_size_ = 10
elif INDEX == 'STOCK':
    lev_ = 5
    slippage_ = 0.001
    lot_size_ = 1
# bnf_pandas = pd.read_csv("../data/finnifty_1hr_tv (2).csv")
# bnf_pandas = pd.read_csv('../data/midcp_1hr_tv (4).csv')
# bnf_pandas = pd.read_csv('../data/finnifty_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/bnf_fut_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')


In [31]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["time"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2015]
bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [32]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

<class 'polars.dataframe.frame.DataFrame'>


In [33]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


In [34]:
# Parameters
ema_short_period = 9
ema_long_period = 21
rsi_period = 14
rsi_short_threshold = 50
rsi_oversold = 30
# stop_loss_pct = 0.5 / 100  # 0.5%
# take_profit_pct = 1.0 / 100  # 1.0%

In [35]:
df = bnf_pandas

In [36]:
# df["ema_short"] = df["close"].ewm(span=ema_short_period, adjust=False).mean()
# df["ema_long"] = df["close"].ewm(span=ema_long_period, adjust=False).mean()

In [37]:
# Calculate RSI
# delta = df["close"].diff()
# gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
# loss = -delta.where(delta < 0, 0).rolling(window=rsi_period).mean()
# rs = gain / loss
# df["rsi"] = 100 - (100 / (1 + rs))

In [38]:
def short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold):

    df["ema_short"] = df["close"].ewm(span=ema_short_period, adjust=False).mean()
    df["ema_long"] = df["close"].ewm(span=ema_long_period, adjust=False).mean()

    # Calculate RSI
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=rsi_period).mean()
    rs = gain / loss
    df["rsi"] = 100 - (100 / (1 + rs))
    
    # Initialize variables
    position = None  # None, 'short'
    tradebook = []
    signal_time = None
    signal_price = None
    entry_time = None
    entry_price = None
    initial_sl = None
    
    # Iterate through rows to generate signals
    for i in range(len(df)):
        row = df.iloc[i]
    
        # Check for short signal condition
        if position is None:
            if row["ema_short"] < row["ema_long"] and row["rsi"] < rsi_short_threshold:
                if signal_price is None:  # New signal
                    signal_time = row["datetime"]
                    signal_price = row["low"]
                else:
                    if row['low'] >= signal_price:
                        # Update Signal Price to Better Candle low
                        signal_time = row["datetime"]
                        signal_price = row["low"]
    
            if signal_price is not None:
                if row["low"] < signal_price and row['high'] >= signal_price:  # Signal broken
                    entry_time = row["datetime"]
                    entry_price = signal_price
                    initial_sl = max(df["high"][max(0, i - 3):i].max(), row["high"])
                    position = "short"
        
                    # Add trade to tradebook
                    tradebook.append({
                        "Signal Time": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": None,
                        "Exit Price": None,
                        "Points Captured": None
                    })
                    signal_time, signal_price = None, None  # Reset signal
                else:
                    signal_price = None
                    if row["ema_short"] < row["ema_long"] and row["rsi"] < rsi_short_threshold:
                        if signal_price is None:  # New signal
                            signal_time = row["datetime"]
                            signal_price = row["low"]
                        else:
                            if row['low'] >= signal_price:
                                # Update Signal Price to Better Candle low
                                signal_time = row["datetime"]
                                signal_price = row["low"]
    
        # Exit logic for short position
        if position is not None:
            if row['close'] >= initial_sl:
                exit_time = row["datetime"]
                exit_price = row['close']
                points_captured = entry_price - exit_price
    
                # Update tradebook
                tradebook[-1]["Exit Time"] = exit_time
                tradebook[-1]["Exit Price"] = exit_price
                tradebook[-1]["Points Captured"] = points_captured
                tradebook[-1]["Remark"] = 'Initial SL Hit'
    
                # Reset position
                position = None
                entry_time, entry_price, initial_sl = None, None, None
            elif row["rsi"] > rsi_oversold:
                exit_time = row["datetime"]
                exit_price = row["close"]
                points_captured = entry_price - exit_price
    
                # Update tradebook
                tradebook[-1]["Exit Time"] = exit_time
                tradebook[-1]["Exit Price"] = exit_price
                tradebook[-1]["Points Captured"] = points_captured
                tradebook[-1]["Remark"] = 'RSI Oversold'
    
                # Reset position
                position = None
                entry_time, entry_price, initial_sl = None, None, None
    
    # Convert tradebook to a DataFrame
    tradebook_df = pd.DataFrame(tradebook)
    return tradebook_df
    # Output
    # print(tradebook_df)
    
    # Optional: Save to CSV
    # tradebook_df.to_csv("tradebook.csv", index=False)


In [39]:
ema_short_period = 7
ema_long_period = 17
rsi_period = 12
rsi_short_threshold = 70
rsi_oversold = 20
tradebook_df = short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold)
# tradebook_df

In [40]:
tradebook_df['Qty'] = 5 * 10000000 / tradebook_df['Entry Price'] #5x Lev, 1Cr portfolio
tradebook_df['PnL'] = tradebook_df['Points Captured'] * tradebook_df['Qty']
tradebook_df['PnL w cs'] = (tradebook_df['Points Captured'] - slippage_*(tradebook_df['Entry Price'] + tradebook_df['Exit Price'])) * tradebook_df['Qty']
tradebook_df['ROI%'] = tradebook_df['PnL w cs'] * 100 / 10000000

In [41]:
tradebook_df['Trade Year'] = tradebook_df['Entry Time'].dt.year

In [42]:
tradebook_df['ROI%'].sum()

-204.56053722838482

In [43]:
tradebook_df[tradebook_df['Trade Year'] >= 2024]

,Signal Time,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Points Captured,Remark,Qty,PnL,PnL w cs,ROI%,Trade Year
1863,2023-12-29 15:15:00,2024-01-01 09:15:00,72152.0600,72350.0100,2024-01-01 09:15:00,72103.4600,48.6000,RSI Oversold,692.9809,33678.8721,23682.2400,0.2368,2024
1864,2024-01-01 09:45:00,2024-01-01 10:00:00,72101.3100,72222.7200,2024-01-01 10:00:00,72115.5900,-14.2800,RSI Oversold,693.4687,-9902.7327,-19903.7230,-0.1990,2024
1865,2024-01-01 12:15:00,2024-01-01 12:30:00,72133.2000,72225.9100,2024-01-01 12:30:00,72189.3700,-56.1700,RSI Oversold,693.1621,-38934.9149,-48938.8083,-0.4894,2024
1866,2024-01-02 09:30:00,2024-01-02 09:45:00,72111.1100,72301.7500,2024-01-02 09:45:00,71866.8500,244.2600,RSI Oversold,693.3744,169363.6390,159380.5753,1.5938,2024
1867,2024-01-02 10:00:00,2024-01-02 10:15:00,71844.0000,72221.3700,2024-01-02 10:15:00,71781.8500,62.1500,RSI Oversold,695.9523,43253.4380,33257.7633,0.3326,2024
1868,2024-01-02 10:45:00,2024-01-02 11:00:00,71695.2700,71965.2800,2024-01-02 11:00:00,71698.8000,-3.5300,RSI Oversold,697.3961,-2461.8081,-12462.0543,-0.1246,2024
1869,2024-01-02 12:30:00,2024-01-02 12:45:00,71793.4100,71857.7700,2024-01-02 12:45:00,71782.1500,11.2600,RSI Oversold,696.4428,7841.9454,-2157.2704,-0.0216,2024
1870,2024-01-02 13:45:00,2024-01-02 14:00:00,71760.5600,71898.0800,2024-01-02 14:00:00,71882.9800,-122.4200,RSI Oversold,696.7616,-85297.5506,-95306.0804,-0.9531,2024
1871,2024-01-02 14:45:00,2024-01-02 15:00:00,71872.2000,71986.2200,2024-01-02 15:00:00,71896.1100,-23.9100,RSI Oversold,695.6793,-16633.6915,-26635.3548,-0.2664,2024
1872,2024-01-03 09:15:00,2024-01-03 09:30:00,71620.6300,71986.2200,2024-01-03 09:30:00,71504.2800,116.3500,RSI Oversold,698.1229,81226.5963,71234.7190,0.7123,2024


In [48]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2021, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2021, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [49]:
variation = f'...{rsi_oversold}'
stats = generate_stats(tradebook_df, variation)

In [50]:
stats_dictionary = {}
for overall_roi_dd_ratio, stats_df in stats.items():
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
        print(stats_df.to_string())
        stats_dictionary[overall_roi_dd_ratio] = stats_df

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2021       1.7593          103  43.6893                0.8461             -0.6261      -7.2320       0.2433     ...30
2022     -25.3685          320  43.1250                0.7324             -0.6947     -39.7378      -0.6384     ...30
2023      18.3065          307  43.3225                0.5830             -0.3404      -9.7663       1.8745     ...30
2024      21.5052          259  44.4015                0.8289             -0.5126     -17.7216       1.2135     ...30
Overall   16.2025          989  43.5794                0.7239             -0.5301     -39.7378       0.4077     ...30


In [51]:
# tradebook_df.to_csv('SENSEX_SHORT.csv', index=False)

In [ ]:
stats_dictionary = {}

for ema_short_period in range(3, 16, 2):
    for ema_long_period in range(5, 24, 2):
        for rsi_period in range(2, 21, 2):
            for rsi_short_threshold in range(20, 81, 10):
                for rsi_oversold in range(20, 61, 10):
                    if (ema_short_period < ema_long_period) and (rsi_oversold <= rsi_short_threshold):
                        tradebook_df = short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold)
                        tradebook_df['Qty'] = 5 * 10000000 / tradebook_df['Entry Price'] #5x Lev, 1Cr portfolio
                        tradebook_df['PnL'] = tradebook_df['Points Captured'] * tradebook_df['Qty']
                        tradebook_df['PnL w cs'] = (tradebook_df['Points Captured'] - slippage_*(tradebook_df['Entry Price'] + tradebook_df['Exit Price'])) * tradebook_df['Qty']
                        tradebook_df['ROI%'] = tradebook_df['PnL w cs'] * 100 / 10000000
                        # tradebook_df['ROI%'] = tradebook_df['PnL'] * 100 / 10000000
                        tradebook_df['Trade Year'] = tradebook_df['Entry Time'].dt.year
                        variation = f'{ema_short_period}, {ema_long_period}, {rsi_period}, {rsi_short_threshold}, {rsi_oversold} -> {round(tradebook_df["ROI%"].sum())}%'
                        print(variation)
                        new_tb = tradebook_df
                        if len(new_tb) > 1:
                            new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                            stats = generate_stats(new_tb, variation)
                            for overall_roi_dd_ratio, stats_df in stats.items():
                                if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 4:
                                    print(stats_df.to_string())
                                    stats_dictionary[overall_roi_dd_ratio] = stats_df

3, 5, 2, 20, 20 -> -202%
3, 5, 2, 30, 20 -> -192%
3, 5, 2, 30, 30 -> -211%
3, 5, 2, 40, 20 -> -188%
3, 5, 2, 40, 30 -> -209%
3, 5, 2, 40, 40 -> -169%
3, 5, 2, 50, 20 -> -207%
3, 5, 2, 50, 30 -> -211%
3, 5, 2, 50, 40 -> -179%
3, 5, 2, 50, 50 -> -151%
3, 5, 2, 60, 20 -> -206%
3, 5, 2, 60, 30 -> -214%
3, 5, 2, 60, 40 -> -197%
3, 5, 2, 60, 50 -> -164%
3, 5, 2, 60, 60 -> -123%
3, 5, 2, 70, 20 -> -201%
3, 5, 2, 70, 30 -> -217%
3, 5, 2, 70, 40 -> -200%
3, 5, 2, 70, 50 -> -162%
3, 5, 2, 70, 60 -> -122%
3, 5, 2, 80, 20 -> -214%
3, 5, 2, 80, 30 -> -226%
3, 5, 2, 80, 40 -> -208%
3, 5, 2, 80, 50 -> -164%
3, 5, 2, 80, 60 -> -121%
3, 5, 4, 20, 20 -> -112%
3, 5, 4, 30, 20 -> -173%
3, 5, 4, 30, 30 -> -87%
3, 5, 4, 40, 20 -> -219%
3, 5, 4, 40, 30 -> -99%
3, 5, 4, 40, 40 -> -63%
3, 5, 4, 50, 20 -> -253%
3, 5, 4, 50, 30 -> -133%
3, 5, 4, 50, 40 -> -93%
3, 5, 4, 50, 50 -> -113%
3, 5, 4, 60, 20 -> -261%
3, 5, 4, 60, 30 -> -137%
3, 5, 4, 60, 40 -> -103%
3, 5, 4, 60, 50 -> -118%
3, 5, 4, 60, 60 -> -63%
3, 5,

In [214]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

{19.85308271140556:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2015      76.7888          211  54.0284                1.4708   
 2016      52.8717          213  51.1737                1.2200   
 2017       5.0646          169  47.9290                0.7562   
 2018      41.1875          172  54.6512                1.1825   
 2019      40.2082          215  52.0930                1.1319   
 2020     111.8902          173  47.3988                2.8002   
 2021      56.1636          167  52.0958                1.6963   
 2022      15.9933          205  54.6341                1.1804   
 2023      23.5168          194  49.4845                1.0193   
 2024      51.1553          171  50.8772                1.3846   
 Overall  474.8400         1890  51.5344                1.3630   
 
         Avg Loss% per Trade Max Drawdown ROI/DD Ratio  \
 2015                -0.9369     -11.5999       6.6198   
 2016                -0.7702     -19.3146       2.7374   
 2017        